In [ ]:
# Code to reproduce figure 2c in Forlin, Mebrahtu et al

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

from scipy.stats import ttest_ind, mannwhitneyu
from itertools import combinations
from statsmodels.stats.multitest import multipletests



In [ ]:
data_path = #insert data path here
adata = sc.read(data_path)
sc.pp.normalize_total(adata, target_sum = 1e3)


In [ ]:
df_cytos = pd.read_csv('./data/cytokine_genelist.csv', index_col = [0])
hallmark_ifna = ast.literal_eval(df_cytos.loc[7, 'genes'])
hallmark_ifnb = ast.literal_eval(df_cytos.loc[8, 'genes'])



In [ ]:
hallmark_list = [hallmark_ifna, hallmark_ifnb]

for i, hallmark in enumerate(['hallmark_ifna', 'hallmark_ifnb']):
    genes = hallmark_list[i]
    df_bootstrapped = pd.DataFrame(columns = ['mean_value', 'CellType', 'treatment'])
    for st in list(set(adata.obs.stim_treat_time)):
        temp = adata[adata.obs.stim_treat_time == st]
        for ct in ['B', 'CD4T', 'CD8T', 'Monocyte', 'DC', 'Eosinophil', 'NK', 'Neutrophil', 'pDC', 'Plasmablast']:
            temp2 = temp[temp.obs.CellType == ct]
            temp2 = temp2[:, temp2.var.index.isin(genes)].to_df()
            
            summ = np.sum(temp2, axis = 1)
            df_ctrl = adata[(adata.obs.stim_treat_time == 'IFNb_Ctrl_2h')&(adata.obs.CellType == ct), adata.var.index.isin(genes)].to_df()
            ctrl_summ = np.sum(df_ctrl, axis = 1)
            
            for i in range(100):
                mean_value = np.mean(summ.sample(len(summ), replace = True))
                df_bootstrapped.loc[st + '_' + ct + '_' + str(i)] = [mean_value, ct, st]
    df_bootstrapped = df_bootstrapped.reset_index().drop('index', axis = 1)
    df_bootstrapped['hallmark'] = hallmark
    #df_bootstrapped.to_csv(f'./data/Figure2_{hallmark}_toR.csv', sep = ',')
            

In [ ]:
df_bootstrapped

In [ ]:
#Get significance
results = []

# Group by hallmark and cell type
for hallmark in df_bootstrapped['hallmark'].unique():
    df_h = df_bootstrapped[df_bootstrapped['hallmark'] == hallmark]
    
    for cell in df_h['CellType'].unique():
        df_c = df_h[df_h['CellType'] == cell]
        treatments = df_c['treatment'].unique()
        
        for t1, t2 in combinations(treatments, 2):
            values1 = df_c[df_c['treatment'] == t1]['mean_value']
            values2 = df_c[df_c['treatment'] == t2]['mean_value']
            
            stat, pval = mannwhitneyu(values1, values2, alternative='two-sided')
            
            results.append({
                'hallmark': hallmark,
                'CellType': cell,
                'treatment_1': t1,
                'treatment_2': t2,
                'statistic': stat,
                'p_value': pval
            })

df_results = pd.DataFrame(results)

# Multiple testing correction
df_results['p_adj'] = multipletests(df_results['p_value'], method='bonferroni')[1]



In [ ]:
df_results